In [4]:
import pandas as pd

In [5]:
# Extract Features
def count_packet(df):
    return len(df)

def total_length(df):
    return df["Length"].sum()

def average_packet_interval(df):
    df = df.sort_values("Time")
    return df["Time"].diff().mean()

def max_packet_interval(df):
    df = df.sort_values("Time")
    return df["Time"].diff().max()

def min_packet_interval(df):
    df = df.sort_values("Time")
    return df["Time"].diff().min()

def average_packet_length(df):
    return df["Length"].mean()

def max_packet_length(df):
    return df["Length"].max()

def min_packet_length(df):
    return df["Length"].min()

def most_common_packet_length(df):
    return df["Length"].mode().iloc[0] if not df["Length"].mode().empty else None


def get_results(df, label="ChatGPT"): # Sign Labels based on actual Website Name
    return {
        "Packet Count": count_packet(df),
        "Total Length": total_length(df),
        "Avg Interval (s)": average_packet_interval(df),
        "Max Interval (s)": max_packet_interval(df),
        "Min Interval (s)": min_packet_interval(df),
        "Avg Length (bytes)": average_packet_length(df),
        "Max Length (bytes)": max_packet_length(df),
        "Min Length (bytes)": min_packet_length(df),
        "Most Common Length (bytes)": most_common_packet_length(df),
        "Label": label
    }

# Build Files 
def process_files(file_prefix="chatGPTdata", file_range=range(1, 21)): # used actual data prefix
    total_df = pd.DataFrame()
    for i in file_range:
        df = pd.read_csv(f"{file_prefix}{i}.csv")
        results = get_results(df)
        total_df = pd.concat([total_df, pd.DataFrame([results])], ignore_index=True)
    return total_df

# Run code
if __name__ == "__main__":
    summary_df = process_files()
    print(summary_df)
    summary_df.to_csv("chatGPT_summary.csv", index=False)
    

FileNotFoundError: [Errno 2] No such file or directory: 'chatGPTdata1.csv'

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Load CSVs
csv1 = pd.read_csv('chatGPT_summary.csv')
csv2 = pd.read_csv('linkedin_summary.csv')
csv3 = pd.read_csv('reddit_summary.csv')
csv4 = pd.read_csv('wikipedia_summary_updated.csv')

# Ensure data types are consistent
csv3['Min Interval (s)'] = csv3['Min Interval (s)'].astype(float)

# Combine and shuffle
combined_df = pd.concat([csv1, csv2, csv3, csv4], ignore_index=True)
combined_df = combined_df.sample(frac=1, random_state=123).reset_index(drop=True)

# Stage 1: split off 60% training
train_df, temp_df = train_test_split(combined_df, test_size=0.4, random_state=42)

# Stage 2: split remaining 40% into 50/50 for validation and test (20% each)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Optional: ensure correct dtypes again
for df in [train_df, val_df, test_df]:
    df['Min Interval (s)'] = df['Min Interval (s)'].astype(float)

# Save to CSV
train_df.to_csv('train.csv', index=False)
val_df.to_csv('validation.csv', index=False)
test_df.to_csv('test.csv', index=False)

# Confirm output shapes
print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")


Train shape: (51, 10)
Validation shape: (17, 10)
Test shape: (17, 10)
